In [ ]:
!pip install python-telegram-bot aiohttp pyngrok
!git clone https://github.com/dhaneshsaini/Fooocus.git
%cd /content/Fooocus
!pip install -r requirements_versions.txt


In [ ]:
# Run Fooocus in API mode
!python entry_with_update.py --share --always-high-vram --listen --api --port 7865 &

In [ ]:
from pyngrok import ngrok

# Start ngrok tunnel to port 7865
public_url = ngrok.connect(7865)
print("Fooocus API URL:", public_url)

# Save this URL for the bot
FOOOCUS_API = str(public_url)

In [ ]:
import aiohttp
import io
import base64
from telegram import Update
from telegram.ext import Application, CommandHandler, ContextTypes

# Your Telegram bot token
TOKEN = "8079802285:AAGHjLH6VFBGl2ulzHR4OloAhIuFCmObbv0"

# --- Fetch all LoRAs ---
async def get_loras():
    async with aiohttp.ClientSession() as session:
        async with session.get(f"{FOOOCUS_API}/sdapi/v1/loras") as resp:
            data = await resp.json()
            return {l["name"]: l for l in data}

# --- Generate Image ---
async def generate_image(prompt, negative, loras_payload):
    payload = {
        "prompt": prompt,
        "negative_prompt": negative,
        "loras": loras_payload,
        "width": 512,
        "height": 512,
        "steps": 25
    }
    async with aiohttp.ClientSession() as session:
        async with session.post(f"{FOOOCUS_API}/sdapi/v1/txt2img", json=payload) as resp:
            data = await resp.json()
            img_data = base64.b64decode(data["images"][0])
            return io.BytesIO(img_data)

# --- /generate ---
async def generate(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        if not context.args:
            await update.message.reply_text("⚠️ Usage: /generate <prompt> | <negative> | <Lora:val,...>")
            return

        parts = " ".join(context.args).split("|")
        prompt = parts[0].strip()
        negative = parts[1].strip() if len(parts) > 1 else "ugly, blurry"
        lora_str = parts[2].strip() if len(parts) > 2 else ""

        all_loras = await get_loras()
        chosen_loras = []
        if lora_str:
            for item in lora_str.split(","):
                if ":" in item:
                    name, val = item.split(":")
                    name, val = name.strip(), float(val.strip())
                    if name in all_loras:
                        chosen_loras.append({"name": name, "value": val})

        img_bytes = await generate_image(prompt, negative, chosen_loras)

        await update.message.reply_photo(
            photo=img_bytes,
            caption=f"✅ Prompt: `{prompt}`\n📉 Negative: `{negative}`\n🎨 LoRAs: {chosen_loras}",
            parse_mode="Markdown"
        )
    except Exception as e:
        await update.message.reply_text(f"⚠️ Error: {e}")

# --- /loras ---
async def loras(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        all_loras = await get_loras()
        msg = "**🎨 Available LoRAs:**\n"
        for name in all_loras.keys():
            msg += f"- `{name}`\n"
        await update.message.reply_text(msg, parse_mode="Markdown")
    except Exception as e:
        await update.message.reply_text(f"⚠️ Error fetching LoRAs: {e}")

# --- /help ---
async def help_cmd(update: Update, context: ContextTypes.DEFAULT_TYPE):
    help_text = (
        "🤖 *Fooocus Bot Commands:*\n\n"
        "/generate <prompt> | <negative> | <Lora:val,...>\n"
        " Example: `/generate a cyberpunk girl | blurry | RealSkin:0.7,AnimeStyle:0.9`\n\n"
        "/loras - List all available LoRAs\n"
        "/help - Show this help message"
    )
    await update.message.reply_text(help_text, parse_mode="Markdown")

# --- Run ---
def main():
    app = Application.builder().token(TOKEN).build()
    app.add_handler(CommandHandler("generate", generate))
    app.add_handler(CommandHandler("loras", loras))
    app.add_handler(CommandHandler("help", help_cmd))
    app.run_polling()

main()